   #     Chapter 2 
    

In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("c2").getOrCreate()

In [22]:
df_flights = spark\
.read\
.option('inferSchema','true')\
.option('header','true')\
.csv('/root/golive/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv')

df_flights.describe()

DataFrame[summary: string, DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: string]

In [33]:
df_flights.explain()

== Physical Plan ==
FileScan csv [DEST_COUNTRY_NAME#184,ORIGIN_COUNTRY_NAME#185,count#186] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/root/golive/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [23]:
df_flights.sort('DEST_COUNTRY_NAME').explain()

# Default Shuffle : 200 
# +- Exchange rangepartitioning(DEST_COUNTRY_NAME#184 ASC NULLS FIRST, 200), true, [id=#204]

== Physical Plan ==
*(1) Sort [DEST_COUNTRY_NAME#184 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(DEST_COUNTRY_NAME#184 ASC NULLS FIRST, 200), true, [id=#204]
   +- FileScan csv [DEST_COUNTRY_NAME#184,ORIGIN_COUNTRY_NAME#185,count#186] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/root/golive/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [32]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

df_flights.sort('ORIGIN_COUNTRY_NAME').explain()

== Physical Plan ==
*(1) Sort [ORIGIN_COUNTRY_NAME#185 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(ORIGIN_COUNTRY_NAME#185 ASC NULLS FIRST, 5), true, [id=#276]
   +- FileScan csv [DEST_COUNTRY_NAME#184,ORIGIN_COUNTRY_NAME#185,count#186] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/root/golive/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [34]:
df_flights.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [36]:
df_flights.limit(5)
df_flights.limit(5).collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [37]:
df_flights.sort('count').explain()

== Physical Plan ==
*(1) Sort [count#186 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#186 ASC NULLS FIRST, 5), true, [id=#309]
   +- FileScan csv [DEST_COUNTRY_NAME#184,ORIGIN_COUNTRY_NAME#185,count#186] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/root/golive/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [40]:
from pyspark.sql.functions import col 
df_flights.select(col('ORIGIN_COUNTRY_NAME')).show(2)
df_flights.select('ORIGIN_COUNTRY_NAME').show(2)

+-------------------+
|ORIGIN_COUNTRY_NAME|
+-------------------+
|            Romania|
|            Croatia|
+-------------------+
only showing top 2 rows

+-------------------+
|ORIGIN_COUNTRY_NAME|
+-------------------+
|            Romania|
|            Croatia|
+-------------------+
only showing top 2 rows



In [41]:
df_flights.show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows



In [42]:
spark.conf.set("spark.sql.shuffle.partitions","5")

In [44]:
df_flights.createOrReplaceTempView('flights_2015')

In [47]:
v_sql = """select DEST_COUNTRY_NAME , count(*) as c from flights_2015 group by DEST_COUNTRY_NAME  """
spark.sql(v_sql).take(10)


[Row(DEST_COUNTRY_NAME='Moldova', c=1),
 Row(DEST_COUNTRY_NAME='Bolivia', c=1),
 Row(DEST_COUNTRY_NAME='Algeria', c=1),
 Row(DEST_COUNTRY_NAME='Turks and Caicos Islands', c=1),
 Row(DEST_COUNTRY_NAME='Pakistan', c=1),
 Row(DEST_COUNTRY_NAME='Marshall Islands', c=1),
 Row(DEST_COUNTRY_NAME='Suriname', c=1),
 Row(DEST_COUNTRY_NAME='Panama', c=1),
 Row(DEST_COUNTRY_NAME='New Zealand', c=1),
 Row(DEST_COUNTRY_NAME='Liberia', c=1)]

In [48]:
spark.sql(v_sql).explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#184], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#184, 5), true, [id=#449]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#184], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#184] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/root/golive/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [43]:
df_flights.groupBy('DEST_COUNTRY_NAME')\
.count().explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#229], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#229, 5), true, [id=#450]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#229], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#229] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/root/golive/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [49]:
v_sql2 = "select max(count) from flights_2015 "
spark.sql(v_sql2).take(1)

spark.sql(v_sql2).explain()

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[max(count#186)])
+- Exchange SinglePartition, true, [id=#492]
   +- *(1) HashAggregate(keys=[], functions=[partial_max(count#186)])
      +- FileScan csv [count#186] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/root/golive/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<count:int>




In [50]:
from pyspark.sql.functions import max 
df_flights.select(max('count')).take(1)

[Row(max(count)=370002)]

In [54]:
v_sql3 = "select DEST_COUNTRY_NAME, sum(count) as ss from flights_2015 group by DEST_COUNTRY_NAME order by ss desc LIMIT 5  "
spark.sql(v_sql3).show()

+-----------------+------+
|DEST_COUNTRY_NAME|    ss|
+-----------------+------+
|    United States|411352|
|           Canada|  8399|
|           Mexico|  7140|
|   United Kingdom|  2025|
|            Japan|  1548|
+-----------------+------+



In [53]:
from pyspark.sql.functions import desc 
new_df = df_flights\
.groupBy('DEST_COUNTRY_NAME')\
.sum('count')\
.withColumnRenamed('sum(count)', 'ss')\
.sort(desc('ss'))\
.limit(5)


new_df.show()

+-----------------+------+
|DEST_COUNTRY_NAME|    ss|
+-----------------+------+
|    United States|411352|
|           Canada|  8399|
|           Mexico|  7140|
|   United Kingdom|  2025|
|            Japan|  1548|
+-----------------+------+



In [54]:
new_df.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[ss#539L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#184,ss#539L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#184], functions=[sum(cast(count#186 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#184, 5), true, [id=#591]
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#184], functions=[partial_sum(cast(count#186 as bigint))])
         +- FileScan csv [DEST_COUNTRY_NAME#184,count#186] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/root/golive/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


